# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [ ]:
train_doc = np.random.permutation(open('D:/Upgrade Master Degree/Gesture Recognition/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('D:/Upgrade Master Degree/Gesture Recognition/Project_data/val.csv').readlines())
batch_size = 12 #experiment with the batch size

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
## 2GB graphics car: batch size = 16, 8,  img_size= 64*64
## 4GB graphics car: batch size = 16, 8,  img_size= 128*128, 64*64
## 6GB graphics car: batch size = 32, 16, 8,  img_size= 128*128, 64*64
## 8GB graphics car: batch size = 128, 64, 32, 16, 8, img_size= 128*128, 64*64

# if train_on CPU:
#     use = Physical RAM
# else if train_on GPU:
#     use = Graphics card VRAM

In [ ]:
# function to plot the training/validation accuracies/losses.
from matplotlib import pyplot as plt
def plot(history):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])   
    axes[0].plot(history.history['val_loss'])
    axes[0].grid()
    axes[0].legend(['loss','val_loss'])
    axes[1].plot(history.history['categorical_accuracy'])   
    axes[1].plot(history.history['val_categorical_accuracy'])
    axes[1].grid()
    axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def cropResize(image, y, z):
    h = image.shape[0]
    w = image.shape[1]
    
    # if smaller image crop at center for 120x120
    if w == 160:
        image = image[:120, 20:140]

    # resize every image
    return resize(image, (y,z))

In [ ]:
def normalizeImage(image):
    # applying normalization
    return image/255.0

In [ ]:
def getBatchDataAndBatchLabels(source_path, t, skipRecords, batch_size, video_frames, img_height, img_width):  
    #create a list of image numbers you want to use for a particular video
    img_idx = video_frames
    x = len(img_idx)
    y = img_height
    z = img_width
    
    batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    
    for folderIndex in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folderIndex + skipRecords].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
            image = imread(source_path+'/'+ t[folderIndex + skipRecords].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes
            resized_image = cropResize(image, y, z)
            
            batch_data[folderIndex,idx,:,:,0] = normalizeImage(resized_image[:,:,0]) #normalise and feed in the image
            batch_data[folderIndex,idx,:,:,1] = normalizeImage(resized_image[:,:,1]) #normalise and feed in the image
            batch_data[folderIndex,idx,:,:,2] = normalizeImage(resized_image[:,:,2]) #normalise and feed in the image
            
        batch_labels[folderIndex, int(t[folderIndex + skipRecords].strip().split(';')[2])] = 1
    return batch_data, batch_labels

In [ ]:
def generator(source_path, folder_list, batch_size, video_frames, img_height, img_width):
    print('Source path = ', source_path, '; batch size =', batch_size)

    while True:
        t = np.random.permutation(folder_list)
        totalFoldersCount = len(t)
        num_full_batches = totalFoldersCount//batch_size # calculate the number of full batches
        
        for currentBatchIndex in range(num_full_batches): # we iterate over the number of full batches
            # currentBatchNumber is 0 index basedso skipRecords value will be 0 for first batch,
            # 1 * 50 = 50 for second batch, 2 * 50 = 100 for third batch, and so on
            skipRecords = currentBatchIndex*batch_size
            yield getBatchDataAndBatchLabels(source_path, t, skipRecords, batch_size, video_frames, img_height, img_width)
    
        remainingItems = totalFoldersCount%batch_size
        if remainingItems != 0:
            # Control will reach here only of all full batches are processed
            # so take all remaining items in last batch
            # For last partial batch skipRecords will be fullbatches * batch size
            # e.g. 663 records have 13 full batches and 14th partial batch of size 13
            # skip 50 * 13 = 650, and take next 13
            skipRecords = num_full_batches * batch_size
            yield getBatchDataAndBatchLabels(source_path, t, skipRecords, remainingItems, video_frames, img_height, img_width)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'D:/Upgrade Master Degree/Gesture Recognition/Project_data/train'
val_path ='D:/Upgrade Master Degree/Gesture Recognition/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # length of the video frames
nb_channel = 3

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras import optimizers
from keras.optimizers import Adam
from keras.layers import LSTM
from tensorflow.keras.applications import mobilenet

from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
#write your model here

In [ ]:

def train_model(modelName, model, batch_size, video_frames, img_height, img_width, num_epochs):
    # Now that you have written the model, the next step is to compile the model.
    # When you print the summary of the model, you'll see the total number of parameters you have to train.
    
    optimiser = tf.keras.optimizers.Adam() #write your optimizer
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    
    # Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.    
    train_generator = generator(train_path, train_doc, batch_size, video_frames, img_height, img_width)
    val_generator = generator(val_path, val_doc, batch_size, video_frames, img_height, img_width)
    
    # save model
    model_name = modelName + '_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)
        
    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
    callbacks_list = [checkpoint, LR]
    
    # The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to 
    # decide the number of next() calls it need to make.
    
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1
    
    # Let us now fit the model. This will start training the model and with the help of the checkpoints,
    # you'll be able to save the model at the end of each epoch.
    
    fitted_model = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
    
    return fitted_model

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:

def getConv2DModel(img_idx, y, z):
    x = len(img_idx)
    
    model = Sequential()

    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation='relu'), input_shape=(x,y,z,3)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    model.add(TimeDistributed(Conv2D(32, (2, 2), padding='same',activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    model.add(TimeDistributed(Conv2D(64, (2, 2), padding='same',activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Flatten()))

    model.add(LSTM(256))
    model.add(Dropout(0.25))
    
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(5,activation='softmax'))
    
    return model

In [ ]:
def getConv3DModel_1(img_idx, y, z):
    x = len(img_idx)

    model = Sequential()
    
    model.add(Conv3D(16, (3,3,3), padding='same', input_shape=(x,y,z,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(Conv3D(32, (2,2,2), padding='same'))
    model.add(Activation('relu'))    
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Conv3D(64, (2,2,2), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(Flatten())

    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(5,activation='softmax'))
    
    return model

In [ ]:
# optimiser = #write your optimizer
# model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# print (model.summary())

In [ ]:
# Take below frames from each video of 30 frames
seq_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28]
image_height = 120
image_width = 120
batch_size = 50
num_epochs = 50

In [ ]:
conv2D_model = getConv2DModel(seq_idx, image_height, image_width)
conv2D_fitted_model = train_model("Conv2D", conv2D_model, batch_size, seq_idx, image_height, image_width, num_epochs)

In [ ]:
plot(conv2D_fitted_model)

In [ ]:
conv3D_model_1 = getConv3DModel_1(seq_idx, image_height, image_width)
conv3D_fitted_model_1 = train_model("Conv3D_1", conv3D_model_1, batch_size, seq_idx, image_height, image_width, num_epochs)

In [ ]:
plot(conv2D_fitted_model_1)